In [1]:
import numpy as np
import pandas as pd
from google.colab import drive

In [2]:
# @title Функции

def top_to_user(k: int, n=10) -> pd.DataFrame:
    user_id = k
    correlations_for_user = correlation_matrix[user_id]
    top_10_users = correlations_for_user.sort_values(ascending=False)[1:n+1]
    return top_10_users

def create_tables(n=999999, k=999999) -> pd.DataFrame:

    drive.mount('/content/drive')

    movie_tags = pd.read_csv('/content/drive/MyDrive/ml-latest/genome-scores.csv', low_memory=False)
    movie_tags = movie_tags[movie_tags['movieId'] < n]

    tags = pd.read_csv('/content/drive/MyDrive/ml-latest/genome-tags.csv', low_memory=False)
    movie_tags = pd.merge(movie_tags, tags, on='tagId', how='left')

    movies = pd.read_csv('/content/drive/MyDrive/ml-latest/links.csv', low_memory=False)
    movies = movies[movies['movieId'] < n]

    movie_details = pd.read_csv('/content/drive/MyDrive/ml-latest/movies.csv', low_memory=False)
    movie_details = movie_details[movie_details['movieId'] < n]
    movie_details['genres'] = movie_details['genres'].str.replace('|', ' ', regex=False)
    movie_details['first_genre'] = movie_details['genres'].str.split().str[0]
    movie_details = movie_details[['movieId', 'title', 'genres', 'first_genre']]

    ratings = pd.read_csv('/content/drive/MyDrive/ml-latest/ratings.csv', low_memory=False)
    ratings['date'] = pd.to_datetime(ratings['timestamp'], unit='s').dt.strftime('%Y-%m-%d')
    ratings = ratings[ratings['movieId'] < n]
    ratings = ratings[ratings['userId'] < k]
    ratings = pd.merge(movie_details[['movieId', 'title']], ratings, on='movieId', how='inner')
    ratings = ratings[['userId', 'movieId', 'title', 'rating', 'timestamp', 'date']]

    ratings_matrix = ratings.pivot_table(index='title', columns='userId', values='rating').fillna(0)
    correlation_matrix = ratings_matrix.corr()

    movie_ratings = ratings.groupby('movieId').agg(
        average_rating=('rating', 'mean'),
        vote_count=('rating', 'count')
    ).reset_index()
    movie_ratings = pd.merge(movie_details[['movieId', 'title']], movie_ratings, on='movieId', how='inner')

    user_tags = pd.read_csv('/content/drive/MyDrive/ml-latest/tags.csv', low_memory=False)
    user_tags['date'] = pd.to_datetime(user_tags['timestamp'], unit='s').dt.strftime('%Y-%m-%d')
    user_tags = user_tags[user_tags['movieId'] < n]
    user_tags = user_tags[user_tags['userId'] < k]

    return ratings_matrix, correlation_matrix, tags, movie_tags, movies, movie_details, ratings, movie_ratings, user_tags

In [3]:
ratings_matrix, correlation_matrix, tags, movie_tags, movies, movie_details, ratings, movie_ratings, user_tags = create_tables(n=5000, k=5000)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. **Pearson Correlation + User Ratings** (метрики)

**Что это?**
- **Pearson Correlation** измеряет степень линейной зависимости между двумя переменными. В контексте рекомендательных систем, это может быть использовано для сравнения сходства в рейтингах пользователей.

**Как работает?**
- Сравниваем, насколько схожи рейтинги пользователей для фильмов, чтобы понять, как сильно они похожи в предпочтениях.
- Предполагает линейную зависимость и нормальное распределение переменных.
- Подходит для непрерывных данных.

Пример:

In [4]:
from scipy.stats import pearsonr

# Создаем матрицу оценок пользователей
ratings_matrix = ratings.pivot_table(index='userId', columns='title', values='rating').fillna(0)

# Функция для вычисления коэффициента корреляции между пользователями
def pearson_similarity(user1, user2, ratings_matrix):
    return pearsonr(ratings_matrix.loc[user1], ratings_matrix.loc[user2])[0]

# Пример расчета сходства между двумя пользователями
similarity = pearson_similarity(1, 2, ratings_matrix)
print(f"Pearson similarity between user 1 and user 2: {similarity}")

Pearson similarity between user 1 and user 2: 0.0754199922570331


---

### 2. **Spearman Rank Correlation** (метрика)

**Что это?**
- **Spearman Rank Correlation** — это статистическая мера зависимости между двумя переменными, которая оценивает их ранговые корреляции (не линейные). Это подходит для рекомендательных систем, когда рейтинги не обязательно являются линейными, но важно, как пользователь оценивает фильмы относительно других.


**Как работает?**
- Ранжируем фильмы по оценкам каждого пользователя и сравниваем эти ранги между пользователями.
- Менее чувствителен к выбросам, так как он ранжирует данные, прежде чем вычислять корреляцию.
- Подходит для обычных данных или когда данные ранжируются.

Пример:


In [5]:
from scipy.stats import spearmanr

# Функция для вычисления Spearman similarity между двумя пользователями
def spearman_similarity(user1, user2, ratings_matrix):
    return spearmanr(ratings_matrix.loc[user1], ratings_matrix.loc[user2])[0]

# Пример расчета сходства между двумя пользователями
similarity = spearman_similarity(1, 2, ratings_matrix)
print(f"Spearman similarity between user 1 and user 2: {similarity}")

Spearman similarity between user 1 and user 2: 0.05439154533542441


---

### 3. **K-Nearest Neighbors (KNN) для Пользовательских Оценок** (алгоритм)

**Что это?**
- **K-Nearest Neighbors (KNN)** — алгоритм поиска ближайших соседей. В рекомендательных системах это можно использовать для нахождения пользователей, которые имеют похожие предпочтения.

**Как работает?**
- С помощью KNN находим пользователей, которые ставят похожие оценки тем же фильмам, что и целевой пользователь. Эти пользователи могут быть использованы для рекомендации фильмов.

Пример:

In [6]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(ratings_matrix)

distances, indices = knn.kneighbors([ratings_matrix.iloc[0]])

recommended_users = ratings_matrix.iloc[indices[0]]
print(recommended_users)

title   'Til There Was You (1997)  'burbs, The (1989)  'night Mother (1986)  \
userId                                                                        
1                             0.0                 0.0                   0.0   
182                           0.0                 0.0                   0.0   
2277                          0.0                 0.0                   0.0   
3136                          0.0                 0.0                   0.0   
2527                          0.0                 0.0                   0.0   

title   ...And Justice for All (1979)  1-900 (06) (1994)  \
userId                                                     
1                                 0.0                0.0   
182                               0.0                0.0   
2277                              0.0                0.0   
3136                              0.0                0.0   
2527                              0.0                0.0   

title   10 Things I Hate 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [7]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=5, algorithm='auto', metric='cosine')
knn.fit(ratings_matrix)

def recommend_movies(user_index, ratings_matrix, knn, n_recommendations=10):
    distances, indices = knn.kneighbors([ratings_matrix.iloc[user_index]])

    recommended_movies = set()

    for idx in indices[0]:
        similar_user_ratings = ratings_matrix.iloc[idx]

        movies_not_rated_by_user = similar_user_ratings[similar_user_ratings > 0].index
        for movie in movies_not_rated_by_user:
            if ratings_matrix.iloc[user_index][movie] == 0:
                recommended_movies.add(movie)

    recommended_movies = list(recommended_movies)[:n_recommendations]

    return recommended_movies

recommended_movies = recommend_movies(1, ratings_matrix, knn, n_recommendations=10)

print(f"Recommended movie titles for user: {recommended_movies}")

Recommended movie titles for user: ["Mary Shelley's Frankenstein (Frankenstein) (1994)", 'Rock, The (1996)', 'Crow, The (1994)', 'Remains of the Day, The (1993)', 'Shallow Grave (1994)', 'Desperado (1995)', 'Much Ado About Nothing (1993)', 'Kalifornia (1993)', 'Bad Boys (1995)', 'Species (1995)']


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


---

### 4. **Matrix Factorization (SVD)** (алгоритм)

**Что это?**
- **Matrix Factorization** — это метод разложения матрицы на две матрицы меньшего размера. В контексте рекомендательных систем, матрица оценок (пользователь-фильм) разлагается на матрицы пользовательских и фильмовских латентных факторов. Одним из популярных методов является **SVD (Singular Value Decomposition)**.

**Как работает?**
- Метод разложения помогает найти скрытые паттерны в оценках, и на основе этих паттернов делать рекомендации.

Пример:


In [8]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=10, random_state=42)
svd_matrix = svd.fit_transform(ratings_matrix)

print(svd_matrix)

[[  8.05770179   3.40754628   4.4055451  ...  -1.08442843   0.84903703
    2.98539411]
 [ 14.71436578  17.42964487 -12.9466904  ...  -2.72418992  -2.792869
    0.68305739]
 [  2.91985034   2.00005617   3.1109815  ...  -1.25099935   0.75140823
   -0.15799019]
 ...
 [  9.14406836   3.50094521   0.7759104  ...  -1.12463476   2.17763761
   -2.6132143 ]
 [ 34.26086739  -1.89959333  -8.2110074  ...  16.98149945  -1.75307071
   13.98565232]
 [ 15.25832329   0.75333363  11.59464095 ...   0.92310374   2.39655512
   -1.58490659]]


---

### 5. **User-User Collaborative Filtering** (алгоритм)

**Что это?**
- **User-User Collaborative Filtering** основан на предположении, что пользователи, которые ставят похожие оценки на одни и те же фильмы, будут иметь схожие предпочтения и в будущем.

**Как работает?**
- Мы находим схожих пользователей и рекомендуем фильмы, которые эти пользователи оценили высоко.

Пример:

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_user_similarity(ratings_matrix):
    return cosine_similarity(ratings_matrix)

def recommend_movies_combined(userId, ratings_matrix, user_similarity, n_recommendations=10):
    similar_users = np.argsort(user_similarity[userId])[-11:-1]

    recommended_movies = {}

    for user in similar_users:
        rated_movies = ratings_matrix.iloc[user][ratings_matrix.iloc[user] > 0].index
        for movie in rated_movies:
            if ratings_matrix.loc[userId, movie] == 0:
                similarity_weight = user_similarity[userId, user]
                if movie not in recommended_movies:
                    recommended_movies[movie] = similarity_weight
                else:
                    recommended_movies[movie] += similarity_weight

    if recommended_movies:
        recommended_movies = sorted(recommended_movies.items(), key=lambda x: x[1], reverse=True)
        recommended_movies = [movie[0] for movie in recommended_movies[:n_recommendations]]
    else:
        recommended_movies = []

    return recommended_movies


user_similarity = compute_user_similarity(ratings_matrix)
recommended_movies = recommend_movies_combined(1, ratings_matrix, user_similarity, n_recommendations=10)
print(f"Recommended movies for user 1: {recommended_movies}")

Recommended movies for user 1: ['Aladdin (1992)', 'Batman (1989)', 'Batman Forever (1995)', 'Clear and Present Danger (1994)', 'Die Hard: With a Vengeance (1995)', 'Dumb & Dumber (Dumb and Dumber) (1994)', 'Fugitive, The (1993)', 'Ghost (1990)', 'Interview with the Vampire: The Vampire Chronicles (1994)', 'Jurassic Park (1993)']


---

### 6. **Clustering (K-Means)** (алгоритм)

**Что это?**
- **Clustering** — метод группировки пользователей или фильмов в кластеры на основе схожести их оценок или других характеристик.

**Как работает?**
- Используя **K-Means**, можно разделить пользователей или фильмы на группы. Пользователи в одной группе будут иметь схожие предпочтения, и можно рекомендовать фильмы из той же группы.

Пример:


In [14]:
import numpy as np
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
user_clusters = kmeans.fit_predict(ratings_matrix)

def recommend_from_cluster(user_index, user_clusters, ratings_matrix, n=10):
    user_cluster = user_clusters[user_index]
    cluster_users = [i for i, cluster in enumerate(user_clusters) if cluster == user_cluster]
    rated_movies = ratings_matrix.iloc[user_index].loc[ratings_matrix.iloc[user_index] > 0].index.tolist()

    recommended_movies = []
    movie_ratings = {}

    for idx in cluster_users:
        if idx == user_index:
            continue
        user_ratings = ratings_matrix.iloc[idx].loc[ratings_matrix.iloc[idx] > 0]
        for movie_id, rating in user_ratings.items():
            if movie_id not in rated_movies:
                if movie_id not in movie_ratings:
                    movie_ratings[movie_id] = []
                movie_ratings[movie_id].append(rating)

    movie_avg_ratings = {movie_id: np.mean(ratings) for movie_id, ratings in movie_ratings.items()}
    recommended_movies = sorted(movie_avg_ratings.items(), key=lambda x: x[1], reverse=True)
    recommended_movie_ids = [movie_id for movie_id, rating in recommended_movies[:n]]
    print(f"Top {n} recommended movies: {recommended_movie_ids}")
    return recommended_movie_ids


recommended_movies = recommend_from_cluster(0, user_clusters, ratings_matrix, n=10)

Top 10 recommended movies: ['24 7: Twenty Four Seven (1997)', "Kelly's Heroes (1970)", 'Scandal (1989)', 'Steam: The Turkish Bath (Hamam) (1997)', 'Live Flesh (Carne trémula) (1997)', 'Children of Paradise (Les enfants du paradis) (1945)', 'Lovers on the Bridge, The (Amants du Pont-Neuf, Les) (1991)', "2 ou 3 choses que je sais d'elle (2 or 3 Things I Know About Her) (1967)", 'La Cérémonie (1995)', 'Alamo, The (1960)']


---

### 7. **Latent Factor Models (ALS)** (алгоритм)

**Что это?**
- **Alternating Least Squares (ALS)** — это метод оптимизации, используемый для матричной факторизации. Он особенно эффективен для работы с разреженными матрицами, как в случае с оценками пользователей.

**Как работает?**
- Модель находит скрытые факторы как для пользователей, так и для фильмов, и использует эти скрытые параметры для рекомендации.

Пример:

In [11]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession

# Создаем сессию Spark
spark = SparkSession.builder.master("local").appName("Recommender").getOrCreate()

# Преобразуем pandas DataFrame в Spark DataFrame
ratings_df = spark.createDataFrame(ratings)

# Применяем ALS для обучения модели
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(ratings_df)

# Рекомендации для пользователя
user_recommendations = model.recommendForUserSubset(ratings_df, 10)
user_recommendations.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{3808, 4.6654525...|
|     2|[{4840, 4.914429}...|
|     3|[{4289, 5.7098885...|
|     4|[{3855, 5.2088447...|
|     5|[{4289, 4.6407514...|
|     6|[{4289, 4.8466644...|
|     7|[{3855, 4.1463895...|
|     8|[{3913, 5.816461}...|
|     9|[{3855, 4.5367155...|
|    10|[{4434, 4.205812}...|
|    11|[{4289, 5.110259}...|
|    12|[{4789, 5.254818}...|
|    13|[{3456, 5.921985}...|
|    14|[{2573, 5.2405515...|
|    15|[{4459, 4.474174}...|
|    17|[{3855, 5.62134},...|
|    18|[{4289, 1.8536396...|
|    19|[{4175, 4.344439}...|
|    20|[{4672, 4.1933208...|
|    21|[{4289, 4.695143}...|
+------+--------------------+
only showing top 20 rows

